# Building a Movie Recommendation System in PySpark - Lab Code-along
![images of vhs tapes on shelf](img/movies.jpg)

## Introduction

In this last lab, we will implement a a movie recommendation system using Alternating Least Squares (ALS) in Spark programming environment.<br> Spark's machine learning libraray `ml` comes packaged with a very efficient imeplementation of ALS algorithm. 

The lab will require you to put into pratice your spark programming skills for creating and manipulating pyspark DataFrames. We will go through a step-by-step process into developing a movie recommendation system using ALS and pyspark using the MovieLens Dataset.

Note: You are advised to refer to [PySpark Documentation](http://spark.apache.org/docs/2.2.0/api/python/index.html) heavily for completing this lab as it will introduce a few new methods. 

## Objectives

You will be able to:

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* Run a brief hyper-parameter selection activity through a scalable grid search
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Building a Recommendation System

We have seen how recommender/Recommendation Systems have played an  integral parts in the success of Amazon (Books, Items), Pandora/Spotify (Music), Google (News, Search), YouTube (Videos) etc.  For Amazon these systems bring more than 30% of their total revenues. For Netflix service, 75% of movies that people watch are based on some sort of recommendation.

> The goal of Recommendation Systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer tailored services.

### We sort of get the concept

For online video content services like Netflix and Hulu, the need to build robust movie recommendation systems is extremely important. An example of recommendation system is such as this:

1.    User A watches Game of Thrones and Breaking Bad.
2.    User B performs a search query for Game of Thrones.
3.    The system suggests Breaking Bad to user B from data collected about user A.


This lab will guide you through a step-by-step process into developing such a movie recommendation system. We will use the MovieLens dataset to build a movie recommendation system using the collaborative filtering technique with Spark's Alternating Least Saqures implementation. After building that recommendation system, we will go through the process of adding a new user to the dataset with some new ratings and obtaining new recommendations for that user.

## Will Nightengale like Toy Story?

Collaborative filtering and matrix decomposition allows us to use the history of others ratings, along with the entire community of ratings, to answer that question.

![image1](img/collab.png)


## Person vs vegetable

It's important to realize that there are two sides to recommendation

![image2](img/item_user_based.png)

## Code for model

If we wanted, we could jump to the code right now to make this happen.

But would we understand it?
```python
from pyspark.ml.recommendation import ALS

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

als_model = als.fit(movie_ratings)
```

## Documentation Station

Let's explore the [documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.recommendation) together to maybe get a better idea of what is happening. 

- which parameters make sense?
- which are completely foreign?

## Rank

What's all this rank of the factorization business?<br>
[the source code documentation](https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/mllib/recommendation/ALS.scala) describes that variable as the "Rank of the feature matrices"

## Assumptions

Matrix decomposition is built on the theory that every individual (user, movie) score is actually the **dot product** of two separate vectors:
- user characteristics 
- movie characteristics

Wait, do you mean like gender, whether the movie is sci-fi or action? do we have that data?

![beyonce-gif](img/beyonce.gif)

## The hidden matricies 
![image4](img/matrix_decomp.png)

## Embeddings

Embeddings are low dimensional hidden factors for items and users.

For e.g. say we have 5 dimensional (i.e., **rank** = 5) embeddings for both items and users (5 chosen randomly, this could be any number - as we saw with PCA and dim. reduction).

For user-X & movie-A, we can say those 5 numbers might represent 5 different characteristics about the movie e.g.:

- How much movie-A is political
- How recent is the movie
- How much special effects are in movie A
- How dialogue driven is the movie
- How linear is the narrative in the movie

In a similar way, 5 numbers in the user embedding matrix might represent:

- How much does user-X like sci-fi movies
- How much does user-X like recent movies … and so on.

But we have *no actual idea* what those factors actually represent.

### If we knew the feature embeddings in advance, it would look something like this:

In [2]:
import numpy as np

# the original matrix of rankings
R = np.array([[2, np.nan, np.nan, 1, 4],
              [5, 1, 2, np.nan, 2],
              [3, np.nan, np.nan, 3, np.nan],
              [1, np.nan, 4, 2, 1]])

# users X factors
P = np.array([[-0.63274434,  1.33686735, -1.55128517],
              [-2.23813661,  0.5123861,  0.14087293],
              [-1.0289794,  1.62052691,  0.21027516],
              [-0.06422255,  1.62892864,  0.33350709]])

# factors X items
Q = np.array([[-2.09507374,  0.52351075,  0.01826269],
              [-0.45078775, -0.07334991,  0.18731052],
              [-0.34161766,  2.46215058, -0.18942263],
              [-1.0925736,  1.04664756,  0.69963111],
              [-0.78152923,  0.89189076, -1.47144019]])

What about that `np.nan` in the third row, last column? How will that item be reviewed by that user?

In [5]:
print(P[2])
print(Q.T[:,3])
P[2].dot(Q.T[:,3])

[-1.0289794   1.62052691  0.21027516]
[-1.0925736   1.04664756  0.69963111]


2.9674713072459067

## Wait, I saw a transpose in there - what's the actual formula?

Terms:<br>
$R$ is the full user-item rating matrix

$P$ is a matrix that contains the users and the k factors represented as (user,factor)

$Q^T$ is a matrix that contains the items and the k factors represented as

$r̂_{u,i}$ represents our prediction for the true rating $r_{ui}$ In order to get an individual rating, you must take the dot product of a row of P and a column of Q

for the entire matrix:
$$ R = PQ^T $$ 

or for individual ratings

$$r̂_{u,i}=q_i^⊤p_u $$ 





### Let's get the whole matrix!

In [ ]:
P.dot(Q.T)

### Look at those results

Are they _exactly_ correct?
![check](img/check.gif)

## ALS benefit: Loss Function

The Loss function $L$ can be calculated as:

$$ L = \sum_{u,i ∈ \kappa}(r_{u,i}− q_i^T p_u)^2 + λ( ||q_i||^2 + |p_u||^2)$$

Where $\kappa$ is the set of (u,i) pairs for which $r_{u,i}$ is known.

To avoid overfitting, the loss function also includes a regularization parameter $\lambda$. We will choose a $\lambda$ to minimize the square of the difference between all ratings in our dataset $R$ and our predictions.

There's the **least squares** part of ALS, got it!

## So now we use gradient descent, right?

![incorrect](img/incorrect.gif)

### Here comes the alternating part

ALS alternates between holding the $q_i$'s constant and the $p_u$'s constant. 

While all $q_i$'s are held constant, each $p_u$ is computed by solving the least squared problem.<br>
After that process has taken place, all the $p_u$'s are held constant while the $q_i$'s are altered to solve the least squares problem, again, each independently.<br> 
This process repeats many times until you've reached convergence (ideally).

### Changing Loss function:

First let's assume first the item vectors are fixed, we first solve for the user vectors:

$$p_u=(\sum{r{u,i}\in r_{u*}}{q_iq_i^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{q_{i}}}$$__
Then we hold the user vectors constant and solve for the item vectors

$$q_i=(\sum{r{u,i}\in r_{i*}}{p_up_u^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{p_{u}}}$$__
This process repeats until convergence

# Review
What levers do we have available to adjust?
![lever-choice](img/levers.jpeg)

- Pros and cons of large rank?
- Pros and cons of lambda size?
- Iterations?

# Enough - let's get to the data

### Importing the Data
To begin with:
* initialize a SparkSession object
* import the dataset found at './data/ratings.csv' into a pyspark DataFrame

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [7]:
!ls data/

movies.csv  ratings.csv


In [8]:
!head data/ratings.csv

In [2]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=True,
                               header=True)

In [11]:
!ls -lah data

total 2.9M
drwxr-xr-x 4 jovyan users  128 Nov  8 15:58 .
drwxr-xr-x 8 jovyan users  256 Nov  8 16:47 ..
-rw-r--r-- 1 jovyan users 483K Jul 22 15:00 movies.csv
-rw-r--r-- 1 jovyan users 2.4M Jul 22 15:00 ratings.csv


Check the data types of each of the values to ensure that they are a type that makes sense given the column.

In [12]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



But if they were ever incorrectly assigned, here's how we fix it:

In [ ]:
from pyspark.sql.types import *  # noqa

In [ ]:
schema = StructType(
    [
        StructField('userId', IntegerType()),
        StructField('movieId', IntegerType()),
        StructField('rating', FloatType()),
        StructField('timestamp', LongType()),
    ]
)

In [ ]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=False,
                               schema=schema,
                               header=True)

In [13]:
movie_ratings.persist()

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [14]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [15]:
movie_ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



We aren't going to need the time stamp, so we can go ahead and remove that column.

In [16]:
movie_ratings = movie_ratings.drop('timestamp')

In [17]:
movie_ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Use the randomSplit method on the pyspark DataFrame to separate the dataset into a training and test set
* Import the ALS module from pyspark.ml.recommendation.
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the userCol, itemCol, and ratingCol to the appropriate names given this dataset. Then fit the data to the training set and assign it to a variable model. 

In [18]:
!cat data/ratings.csv | wc -l

100837


In [3]:
# split into training and testing sets
# How would we do that?
train, test = movie_ratings.randomSplit([0.8, 0.2], seed=12345)

In [21]:
test.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     50|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
+------+-------+------+
only showing top 5 rows



In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=20,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

In [5]:
# Build the recommendation model using ALS on the training data
# fit the ALS model to the training set

als_model = als.fit(train)

Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* import `RegressionEvalutor` from pyspark.ml.evaluation ([documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator)
* generate predictions with your model for the test set by using the `transform` method on your ALS model
* evaluate your model and print out the RMSE from your test set [options for evaluating regressors](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator.metricName)

In [ ]:
ALSModel.transform?

In [6]:
predictions = als_model.transform(train)

In [25]:
predictions.persist()

DataFrame[userId: int, movieId: int, rating: double, prediction: float]

In [28]:
train.show(1)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
+------+-------+------+
only showing top 1 row



In [31]:
predictions.filter((predictions.userId == 1) & (predictions.movieId == 1)).show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      1|   4.0| 4.8377624|
+------+-------+------+----------+



In [33]:
predictions.show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   191|    148|   5.0|  4.936124|
|   133|    471|   4.0| 3.1868148|
|   597|    471|   2.0| 3.5453417|
|   385|    471|   4.0|  3.518012|
|    91|    471|   1.0| 2.2328289|
|   409|    471|   3.0| 3.2610219|
|   372|    471|   3.0| 2.7259524|
|   474|    471|   3.0|  3.508135|
|   500|    471|   1.0| 1.8460243|
|    57|    471|   3.0| 3.6946206|
+------+-------+------+----------+
only showing top 10 rows



In [11]:
user_factors = als_model.userFactors

In [12]:
user_factors.toPandas()

id                                           features
0     10  [-1.3052246570587158, -0.8679254651069641, 0.6...
1     20  [-0.17875438928604126, -0.010365867987275124, ...
2     30  [-0.41009432077407837, -0.6152682304382324, 1....
3     40  [-0.7280457019805908, -0.36541345715522766, 0....
4     50  [0.26124700903892517, 0.2156444489955902, 0.39...
..   ...                                                ...
605  569  [-0.8774425387382507, -0.4762774109840393, 0.8...
606  579  [-0.3266938626766205, -0.9494704008102417, 0.8...
607  589  [-0.32554930448532104, -0.4599880874156952, 0....
608  599  [-0.058326978236436844, -0.2504945397377014, 0...
609  609  [-0.15845631062984467, -0.606983482837677, 0.3...

[610 rows x 2 columns]

In [13]:
item_factors = als_model.itemFactors

In [14]:
item_factors

DataFrame[id: int, features: array<float>]

In [15]:
item_factors.toPandas()

id                                           features
0         10  [-0.338405579328537, -0.6912245154380798, 0.65...
1         20  [-0.14953407645225525, -0.7959967255592346, 0....
2         30  [1.3049371242523193, 0.2979934513568878, -0.12...
3         40  [-0.451448917388916, -0.9287986755371094, 0.56...
4         50  [-0.36308392882347107, -0.34057024121284485, 0...
...      ...                                                ...
8978  188189  [1.0124263763427734, -0.20458483695983887, 0.3...
8979  190209  [0.3322949707508087, -0.045351069420576096, 0....
8980  190219  [0.08307374268770218, -0.011337767355144024, 0...
8981  193579  [-0.05305759608745575, -0.33589455485343933, 0...
8982  193609  [0.2264452427625656, -0.4784950017929077, 0.55...

[8983 rows x 2 columns]

### Important Question

Will Billy like movie m?

In [16]:
import numpy as np

In [17]:
billy_row = user_factors[user_factors['id'] == 10].first()
billy_factors = np.array(billy_row['features'])


In [18]:
billy_factors

array([-1.30522466, -0.86792547,  0.68878919, -0.10136885, -0.92634934,
        0.28532508, -0.05555166, -1.09902918, -0.04570713,  0.54491776])

In [19]:
m_row = item_factors[item_factors['id'] == 1].first()
m_factors = np.array(m_row['features'])

In [20]:
billy_factors

array([-1.30522466, -0.86792547,  0.68878919, -0.10136885, -0.92634934,
        0.28532508, -0.05555166, -1.09902918, -0.04570713,  0.54491776])

In [21]:
m_factors

array([-0.50441974, -0.27559286,  0.98691982,  1.17906499, -0.96246517,
       -0.41498956,  0.60531336, -0.74297726, -0.26766858,  0.21476068])

In [22]:
billy_factors @ m_factors

3.143195232467673

In [44]:
billy_factors.dot(m_factors)

2.54289606710549

In [45]:
np.dot(billy_factors, m_factors)

2.54289606710549

In [48]:
!cat data/movies.csv | grep "^296,"

In [49]:
billy_preds = predictions[predictions['userId'] == 10]

In [50]:
billy_preds.sort('movieId').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    10|    296|   1.0| 2.5428958|
|    10|    356|   3.5| 3.6567261|
|    10|    597|   3.5| 3.7147207|
|    10|   1028|   0.5|  2.716481|
|    10|   1247|   3.0| 2.8611476|
|    10|   1307|   3.0| 3.2457325|
|    10|   1784|   3.5| 3.2315855|
|    10|   1907|   4.0| 3.4796221|
|    10|   2762|   0.5| 2.7562406|
|    10|   3882|   3.0| 3.1112702|
|    10|   4246|   3.5|  3.334133|
|    10|   4447|   4.5|  3.754142|
|    10|   4993|   4.0| 3.1021152|
|    10|   4995|   4.0| 3.4510937|
|    10|   5377|   3.5| 3.0860987|
|    10|   5620|   3.0| 3.7636862|
|    10|   5952|   4.0| 3.1044304|
|    10|   6535|   4.0| 3.6522074|
|    10|   7149|   4.0| 3.8141768|
|    10|   7151|   3.0| 2.9932463|
+------+-------+------+----------+
only showing top 20 rows



In [52]:
!grep "^356," < data/movies.csv

In [7]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.5581913796109031


## Okay, what *will* Billy like?

In [ ]:
recs = als_model.recommendForAllUsers(numItems=10)

In [ ]:
recs[recs['userId']==10].first()['recommendations']

In [ ]:
!grep "^112804," < data/movies.csv

## Objective Review

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* Run a brief hyper-parameter selection activity through a scalable grid search
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Some great technical resources:

- [good one from Stanford](http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf)
- [the netflix recommendation project](https://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf)